### Move Cost Optimization

In [6]:
import pandas as pd
import itertools
import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds, linprog
from itertools import product
import torch
import torch.optim as optim

#### Scenario
Suppose that I have a business with three job categories of engineer, baker, and accountant. Each category has five promotion levels of 1 through 5. There are also four city locations where each of these categories of employees can work from. The company allows workers to change locations about every three years and will pay for it, but a level and position must be available. How can I use python with either scipy optimize or pytorch optimize to optimize the number of moves each year by maximizing the available moves while minimizing the total cost to staying within a certain budget?

In [ ]:
df = pd.DataFrame({
    'position': ['engineer', 'scientist', 'accountant'],
    'levels': [1, 2, 3, 4, 5],
    'cities': ['New York', 'Los Angeles', 'Chicago', 'Houston']
})
df

In [13]:
cities = ['Seattle', 'Los Angeles', 'Denver', 'Austin']
job_categories = ['Engineer', 'Scientist', 'Accountant']
levels = [1, 2, 3, 4, 5]

combinations = list(itertools.product(cities, job_categories, levels))

df = pd.DataFrame(combinations, columns=['City', 'Positions', 'Level'])
df['People'] = [100, 200, 200, 50, 5,
               25, 45, 55, 15, 3,
               3, 4, 5, 2, 1,
               200, 300, 350, 150, 15,
               50, 80, 95, 30, 6,
               9, 12, 9, 4, 2,
               80, 150, 150, 50, 5,
               30, 45, 55, 15, 3,
               3, 4, 5, 2, 1,
               120, 130, 140, 80, 10,
               60, 50, 50, 30, 5,
               6, 10, 9, 4, 2,]
df

,City,Positions,Level,People
0,Seattle,Engineer,1,100
1,Seattle,Engineer,2,200
2,Seattle,Engineer,3,200
3,Seattle,Engineer,4,50
4,Seattle,Engineer,5,5
5,Seattle,Scientist,1,25
6,Seattle,Scientist,2,45
7,Seattle,Scientist,3,55
8,Seattle,Scientist,4,15
9,Seattle,Scientist,5,3


In [17]:
df.People.sum() * .2 *8000

5326400.000000001

In [18]:
# Gemini
import numpy as np
import pandas as pd
import itertools
from scipy.optimize import milp, LinearConstraint, Bounds

# 1. Define Problem Parameters from user input
cities = ['Seattle', 'Los Angeles', 'Denver', 'Austin']
job_categories = ['Engineer', 'Scientist', 'Accountant']
levels = [1, 2, 3, 4, 5]

num_jobs = len(job_categories)
num_levels = len(levels)
num_cities = len(cities)

# Create the DataFrame to load the 'People' data
combinations = list(itertools.product(cities, job_categories, levels))
df = pd.DataFrame(combinations, columns=['City', 'Positions', 'Level'])
df['People'] = [
    100, 200, 200, 50, 5,
    25, 45, 55, 15, 3,
    3, 4, 5, 2, 1,
    200, 300, 350, 150, 15,
    50, 80, 95, 30, 6,
    9, 12, 9, 4, 2,
    80, 150, 150, 50, 5,
    30, 45, 55, 15, 3,
    3, 4, 5, 2, 1,
    120, 130, 140, 80, 10,
    60, 50, 50, 30, 5,
    6, 10, 9, 4, 2,
]

# E_jlc: Current number of employees, reshaped for numpy
# The order needs to match the loops later: (jobs, levels, cities)
# We pivot the table to get the right shape
current_employees_df = df.pivot_table(index='City', columns=['Positions', 'Level'], values='People')
current_employees = np.zeros((num_jobs, num_levels, num_cities))

# Map names to indices
city_map = {city: i for i, city in enumerate(cities)}
job_map = {job: i for i, job in enumerate(job_categories)}
level_map = {level: i for i, level in enumerate(levels)}

for city, city_idx in city_map.items():
    for job, job_idx in job_map.items():
        for level, level_idx in level_map.items():
            current_employees[job_idx, level_idx, city_idx] = df[
                (df['City'] == city) & (df['Positions'] == job) & (df['Level'] == level)
            ]['People'].iloc[0]


# --- ASSUMPTIONS FOR MISSING DATA ---
np.random.seed(42)
# Assumption 1: Headcount limits (H_jlc) are slightly higher than current levels
# We'll add a small buffer (e.g., 5-10% of current staff, plus 1) to create open slots.
headcount_limits = np.ceil(current_employees * 1.07).astype(int) + 1

# Assumption 2: Requests (R_jlc1c2)
# Assume about 10% of employees in any position might request a transfer.
# The requests are distributed randomly among other cities.
requests_percentage = 0.10
requests = np.zeros((num_jobs, num_levels, num_cities, num_cities))
for j in range(num_jobs):
    for l in range(num_levels):
        for c1 in range(num_cities):
            total_requests_from_loc = int(current_employees[j, l, c1] * requests_percentage)
            if total_requests_from_loc > 0:
                # Distribute these requests among the other 3 cities
                possible_destinations = [c2 for c2 in range(num_cities) if c1 != c2]
                if possible_destinations: # Check if there are other cities
                    # Generate random request numbers that sum to total_requests_from_loc
                    req_dist = np.random.multinomial(total_requests_from_loc, [1/len(possible_destinations)]*len(possible_destinations))
                    for i, c2 in enumerate(possible_destinations):
                        requests[j, l, c1, c2] = req_dist[i]


# --- Constants from user ---
move_cost = 8000
total_budget = 5000000

# 2. Map problem to MILP format
var_map = {}
idx = 0
for j in range(num_jobs):
    for l in range(num_levels):
        for c1 in range(num_cities):
            for c2 in range(num_cities):
                if c1 == c2: continue
                var_map[(j, l, c1, c2)] = idx
                idx += 1
num_vars = len(var_map)

# Objective function: Minimize Sum(-1 * x)
c = -np.ones(num_vars)

# Bounds for each variable x: 0 <= x <= num_requests
lower_bounds = np.zeros(num_vars)
upper_bounds = np.array([requests[j, l, c1, c2] for (j, l, c1, c2), i in var_map.items()])
bounds = Bounds(lower_bounds, upper_bounds)

# Integrality constraint: all variables must be integers
integrality = np.ones(num_vars)

# Build the constraint matrix A and bounds vector b
constraints = []

# Constraint 1: Budget
budget_coeffs = np.full(num_vars, move_cost)
constraints.append(LinearConstraint(budget_coeffs, -np.inf, total_budget))

# Constraint 2: Headcount
num_headcount_constraints = num_jobs * num_levels * num_cities
headcount_matrix = np.zeros((num_headcount_constraints, num_vars))
headcount_rhs = np.zeros(num_headcount_constraints)

hc_idx = 0
for j in range(num_jobs):
    for l in range(num_levels):
        for c_dest in range(num_cities):
            headcount_rhs[hc_idx] = headcount_limits[j, l, c_dest] - current_employees[j, l, c_dest]
            for (j_var, l_var, c1_var, c2_var), i_var in var_map.items():
                if j_var == j and l_var == l:
                    if c2_var == c_dest: # Arrival
                        headcount_matrix[hc_idx, i_var] = 1
                    if c1_var == c_dest: # Departure
                        headcount_matrix[hc_idx, i_var] = -1
            hc_idx += 1

constraints.append(LinearConstraint(headcount_matrix, -np.inf, headcount_rhs))

# 3. Solve the optimization problem
print("🚀 Optimizing employee moves...")
res = milp(c=c, constraints=constraints, bounds=bounds, integrality=integrality)
print("✅ Optimization finished!")


# 4. Display the results
if res.success:
    print("\n--- ✅ Optimal Move Plan ---")
    total_moves = 0
    total_cost = 0
    approved_moves = res.x

    # Create a list of move details for sorting
    move_details = []
    for (j, l, c1, c2), i in var_map.items():
        num_to_move = int(round(approved_moves[i]))
        if num_to_move > 0:
            cost = move_cost * num_to_move
            total_cost += cost
            total_moves += num_to_move
            move_details.append(
                f"Move {num_to_move} {job_categories[j]}(s) at Level {levels[l]} "
                f"from {cities[c1]} to {cities[c2]}"
            )
    
    # Sort and print the move details for readability
    for detail in sorted(move_details):
        print(detail)

    print("\n--- 📊 Summary ---")
    print(f"Total approved moves: {total_moves}")
    print(f"Total cost: ${total_cost:,.2f}")
    print(f"Budget:     ${total_budget:,.2f}")
    print(f"Budget usage: {total_cost / total_budget:.2%}")
else:
    print("\n--- ❌ Optimization Failed ---")
    print(f"The solver could not find an optimal solution. Status: {res.status}")
    print(f"Message: {res.message}")

🚀 Optimizing employee moves...
✅ Optimization finished!

--- ✅ Optimal Move Plan ---
Move 1 Accountant(s) at Level 2 from Austin to Los Angeles
Move 1 Accountant(s) at Level 2 from Los Angeles to Seattle
Move 1 Engineer(s) at Level 1 from Denver to Seattle
Move 1 Engineer(s) at Level 1 from Seattle to Austin
Move 1 Engineer(s) at Level 4 from Austin to Los Angeles
Move 1 Engineer(s) at Level 4 from Seattle to Austin
Move 1 Engineer(s) at Level 4 from Seattle to Los Angeles
Move 1 Engineer(s) at Level 5 from Austin to Seattle
Move 1 Engineer(s) at Level 5 from Los Angeles to Denver
Move 1 Scientist(s) at Level 1 from Denver to Seattle
Move 1 Scientist(s) at Level 1 from Seattle to Austin
Move 1 Scientist(s) at Level 1 from Seattle to Los Angeles
Move 1 Scientist(s) at Level 2 from Austin to Seattle
Move 1 Scientist(s) at Level 2 from Denver to Austin
Move 1 Scientist(s) at Level 2 from Denver to Seattle
Move 1 Scientist(s) at Level 2 from Seattle to Austin
Move 1 Scientist(s) at Level 3

In [19]:
# Claude
import numpy as np
from scipy.optimize import linprog
import pandas as pd
from itertools import product
import itertools
import torch
import torch.optim as optim

class EmployeeRelocationOptimizer:
    def __init__(self, custom_data=None):
        if custom_data is not None:
            self.df = custom_data
            self.cities = self.df['City'].unique().tolist()
            self.job_categories = self.df['Positions'].unique().tolist()
            self.promotion_levels = sorted(self.df['Level'].unique().tolist())
            self.positions = [(row['City'], row['Positions'], row['Level']) for _, row in self.df.iterrows()]
            self.n_positions = len(self.positions)
            self.current_employees = self.df['People'].values
        else:
            self.job_categories = ['engineer', 'baker', 'accountant']
            self.promotion_levels = [1, 2, 3, 4, 5]
            self.cities = ['city_A', 'city_B', 'city_C', 'city_D']
            
            # Create all possible position combinations
            self.positions = list(product(self.job_categories, self.promotion_levels, self.cities))
            self.n_positions = len(self.positions)
        
    def setup_optimization_data(self, avg_cost=8000):
        """Setup optimization data based on current employee data"""
        if not hasattr(self, 'current_employees'):
            raise ValueError("Employee data not loaded. Initialize with custom_data parameter.")
        
        # Available positions - assume 20% more capacity than current employees
        # This simulates some positions being available for transfers
        self.available_positions = (self.current_employees * 1.2).astype(int)
        
        # Relocation costs - base cost with variations
        base_cost = avg_cost
        self.relocation_costs = np.full(self.n_positions, base_cost, dtype=float)
        
        # Add cost variations based on level and city
        for i, (city, job, level) in enumerate(self.positions):
            # Higher levels cost more (10% per level above 1)
            level_multiplier = 1 + (level - 1) * 0.1
            
            # Different cities have different costs
            city_multipliers = {
                'Seattle': 1.2,      # Tech hub, expensive
                'Los Angeles': 1.15, # High cost of living
                'Denver': 0.9,       # Moderate cost
                'Austin': 0.95       # Growing but still reasonable
            }
            city_multiplier = city_multipliers.get(city, 1.0)
            
            # Engineers might be more expensive to relocate due to specialized equipment/setup
            job_multipliers = {
                'Engineer': 1.1,
                'Scientist': 1.05,
                'Accountant': 1.0
            }
            job_multiplier = job_multipliers.get(job, 1.0)
            
            self.relocation_costs[i] = base_cost * level_multiplier * city_multiplier * job_multiplier
        
        # Employee preferences/eligibility for moves
        self.move_preferences = np.random.uniform(0.3, 0.8, (self.n_positions, self.n_positions))
        # No self-moves
        np.fill_diagonal(self.move_preferences, 0)
        
        return self
        
    def scipy_optimization(self, budget=500000, max_moves_per_employee=1):
        """
        Optimize using scipy linear programming
        Maximize total moves while staying within budget
        """
        print("Optimizing with SciPy Linear Programming...")
        
        # Decision variables: x[i,j] = number of employees moving from position i to position j
        n_vars = self.n_positions * self.n_positions
        
        # Objective: maximize total moves (minimize negative of total moves)
        c = -np.ones(n_vars)  # Negative because linprog minimizes
        
        # Constraints
        A_ub = []  # Inequality constraint coefficients
        b_ub = []  # Inequality constraint bounds
        A_eq = []  # Equality constraint coefficients  
        b_eq = []  # Equality constraint bounds
        
        # Budget constraint: sum of all moves * costs <= budget
        budget_constraint = np.zeros(n_vars)
        for i in range(self.n_positions):
            for j in range(self.n_positions):
                var_idx = i * self.n_positions + j
                budget_constraint[var_idx] = self.relocation_costs[j]
        A_ub.append(budget_constraint)
        b_ub.append(budget)
        
        # Capacity constraints: moves to position j <= available positions in j
        for j in range(self.n_positions):
            capacity_constraint = np.zeros(n_vars)
            for i in range(self.n_positions):
                var_idx = i * self.n_positions + j
                capacity_constraint[var_idx] = 1
            A_ub.append(capacity_constraint)
            b_ub.append(self.available_positions[j])
        
        # Supply constraints: moves from position i <= current employees in i
        for i in range(self.n_positions):
            supply_constraint = np.zeros(n_vars)
            for j in range(self.n_positions):
                var_idx = i * self.n_positions + j
                supply_constraint[var_idx] = 1
            A_ub.append(supply_constraint)
            b_ub.append(self.current_employees[i])
        
        # Bounds: all variables >= 0
        bounds = [(0, None) for _ in range(n_vars)]
        
        # Solve
        result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
        
        if result.success:
            moves_matrix = result.x.reshape(self.n_positions, self.n_positions)
            total_moves = np.sum(moves_matrix)
            total_cost = np.sum(moves_matrix * self.relocation_costs.reshape(-1, 1))
            
            print(f"SciPy Optimization Results:")
            print(f"Total moves: {total_moves:.0f}")
            print(f"Total cost: ${total_cost:,.2f}")
            print(f"Budget utilization: {total_cost/budget*100:.1f}%")
            
            return {
                'moves_matrix': moves_matrix,
                'total_moves': total_moves,
                'total_cost': total_cost,
                'success': True
            }
        else:
            print("SciPy optimization failed:", result.message)
            return {'success': False}
    
    def pytorch_optimization(self, budget=500000, learning_rate=0.01, epochs=1000):
        """
        Optimize using PyTorch with gradient descent
        This treats it as a continuous optimization problem with penalty methods
        """
        print("\nOptimizing with PyTorch...")
        
        # Initialize decision variables (moves matrix)
        moves = torch.rand(self.n_positions, self.n_positions, requires_grad=True)
        
        # Convert data to tensors
        costs = torch.tensor(self.relocation_costs, dtype=torch.float32)
        current_emp = torch.tensor(self.current_employees, dtype=torch.float32)
        available_pos = torch.tensor(self.available_positions, dtype=torch.float32)
        
        optimizer = optim.Adam([moves], lr=learning_rate)
        
        best_moves = None
        best_objective = float('-inf')
        
        for epoch in range(epochs):
            optimizer.zero_grad()
            
            # Ensure non-negative and no self-moves
            moves_pos = torch.relu(moves)
            moves_pos = moves_pos * (1 - torch.eye(self.n_positions))
            
            # Objective: maximize total moves
            total_moves = torch.sum(moves_pos)
            
            # Constraints as penalties
            # Budget constraint
            total_cost = torch.sum(moves_pos * costs.unsqueeze(0))
            budget_penalty = torch.relu(total_cost - budget) * 1000
            
            # Capacity constraints
            moves_to_positions = torch.sum(moves_pos, dim=0)
            capacity_penalty = torch.sum(torch.relu(moves_to_positions - available_pos)) * 100
            
            # Supply constraints  
            moves_from_positions = torch.sum(moves_pos, dim=1)
            supply_penalty = torch.sum(torch.relu(moves_from_positions - current_emp)) * 100
            
            # Combined objective (maximize moves, minimize penalties)
            objective = total_moves - budget_penalty - capacity_penalty - supply_penalty
            loss = -objective  # Minimize negative objective
            
            loss.backward()
            optimizer.step()
            
            if objective.item() > best_objective:
                best_objective = objective.item()
                best_moves = moves_pos.detach().clone()
            
            if epoch % 100 == 0:
                print(f"Epoch {epoch}: Moves={total_moves.item():.1f}, "
                      f"Cost=${total_cost.item():,.0f}, Objective={objective.item():.1f}")
        
        if best_moves is not None:
            final_moves = best_moves.numpy()
            total_moves = np.sum(final_moves)
            total_cost = np.sum(final_moves * self.relocation_costs.reshape(-1, 1))
            
            print(f"\nPyTorch Optimization Results:")
            print(f"Total moves: {total_moves:.0f}")
            print(f"Total cost: ${total_cost:,.2f}")
            print(f"Budget utilization: {total_cost/budget*100:.1f}%")
            
            return {
                'moves_matrix': final_moves,
                'total_moves': total_moves,
                'total_cost': total_cost,
                'success': True
            }
        else:
            return {'success': False}
    
    def analyze_results(self, result):
        """Analyze and display optimization results"""
        if not result['success']:
            print("Optimization failed!")
            return
            
        moves_matrix = result['moves_matrix']
        
        # Find top moves
        print(f"\nTop 10 Recommended Moves:")
        print("=" * 80)
        
        move_list = []
        for i in range(self.n_positions):
            for j in range(self.n_positions):
                if moves_matrix[i, j] > 0.1:  # Only show significant moves
                    from_pos = self.positions[i]
                    to_pos = self.positions[j]
                    move_list.append({
                        'from': f"{from_pos[1]} L{from_pos[2]} ({from_pos[0]})",
                        'to': f"{to_pos[1]} L{to_pos[2]} ({to_pos[0]})",
                        'employees': moves_matrix[i, j],
                        'cost_per_move': self.relocation_costs[j],
                        'total_cost': moves_matrix[i, j] * self.relocation_costs[j]
                    })
        
        # Sort by number of employees
        move_list.sort(key=lambda x: x['employees'], reverse=True)
        
        for i, move in enumerate(move_list[:10]):
            print(f"{i+1:2d}. {move['employees']:4.1f} employees: "
                  f"{move['from']} → {move['to']} "
                  f"(${move['cost_per_move']:,.0f} each, ${move['total_cost']:,.0f} total)")
    
    def compare_methods(self, budget=5000000):
        """Compare both optimization methods"""
        print("Employee Relocation Optimization with Real Data")
        print("=" * 60)
        
        # Setup optimization data
        self.setup_optimization_data()
        
        print(f"Problem Setup:")
        print(f"- Total positions: {self.n_positions}")
        print(f"- Cities: {', '.join(self.cities)}")
        print(f"- Job categories: {', '.join(self.job_categories)}")
        print(f"- Promotion levels: {self.promotion_levels}")
        print(f"- Total current employees: {np.sum(self.current_employees):,}")
        print(f"- Total available positions: {np.sum(self.available_positions):,}")
        print(f"- Budget: ${budget:,}")
        print(f"- Average relocation cost: ${np.mean(self.relocation_costs):,.0f}")
        print(f"- Cost range: ${np.min(self.relocation_costs):,.0f} - ${np.max(self.relocation_costs):,.0f}")
        
        # Show current distribution by city and job
        print(f"\nCurrent Employee Distribution:")
        city_totals = {}
        job_totals = {}
        for i, (city, job, level) in enumerate(self.positions):
            city_totals[city] = city_totals.get(city, 0) + self.current_employees[i]
            job_totals[job] = job_totals.get(job, 0) + self.current_employees[i]
        
        print("By City:")
        for city, total in city_totals.items():
            print(f"  {city}: {total:,} employees")
        
        print("By Job Category:")
        for job, total in job_totals.items():
            print(f"  {job}: {total:,} employees")
        
        # SciPy optimization
        scipy_result = self.scipy_optimization(budget)
        
        # PyTorch optimization
        pytorch_result = self.pytorch_optimization(budget)
        
        # Analysis
        if scipy_result['success']:
            print(f"\n" + "="*50)
            print("SCIPY RESULTS ANALYSIS:")
            self.analyze_results(scipy_result)
        
        if pytorch_result['success']:
            print(f"\n" + "="*50)  
            print("PYTORCH RESULTS ANALYSIS:")
            self.analyze_results(pytorch_result)

# Example usage with your specific data
if __name__ == "__main__":
    # Create the dataframe with your specific data
    cities = ['Seattle', 'Los Angeles', 'Denver', 'Austin']
    job_categories = ['Engineer', 'Scientist', 'Accountant']
    levels = [1, 2, 3, 4, 5]
    combinations = list(itertools.product(cities, job_categories, levels))
    df = pd.DataFrame(combinations, columns=['City', 'Positions', 'Level'])
    df['People'] = [100, 200, 200, 50, 5,
                   25, 45, 55, 15, 3,
                   3, 4, 5, 2, 1,
                   200, 300, 350, 150, 15,
                   50, 80, 95, 30, 6,
                   9, 12, 9, 4, 2,
                   80, 150, 150, 50, 5,
                   30, 45, 55, 15, 3,
                   3, 4, 5, 2, 1,
                   120, 130, 140, 80, 10,
                   60, 50, 50, 30, 5,
                   6, 10, 9, 4, 2,]
    
    print("Employee Data Overview:")
    print("=" * 40)
    print(df.head(10))
    print(f"\nTotal employees: {df['People'].sum():,}")
    print(f"Data shape: {df.shape}")
    
    # Create optimizer with your data
    optimizer = EmployeeRelocationOptimizer(custom_data=df)
    
    # Run optimization with $5M budget and $8K average cost
    optimizer.compare_methods(budget=5000000)

Employee Data Overview:
      City  Positions  Level  People
0  Seattle   Engineer      1     100
1  Seattle   Engineer      2     200
2  Seattle   Engineer      3     200
3  Seattle   Engineer      4      50
4  Seattle   Engineer      5       5
5  Seattle  Scientist      1      25
6  Seattle  Scientist      2      45
7  Seattle  Scientist      3      55
8  Seattle  Scientist      4      15
9  Seattle  Scientist      5       3

Total employees: 3,329
Data shape: (60, 4)
Employee Relocation Optimization with Real Data
Problem Setup:
- Total positions: 60
- Cities: Seattle, Los Angeles, Denver, Austin
- Job categories: Engineer, Scientist, Accountant
- Promotion levels: [1, 2, 3, 4, 5]
- Total current employees: 3,329
- Total available positions: 3,984
- Budget: $5,000,000
- Average relocation cost: $10,584
- Cost range: $7,200 - $14,784

Current Employee Distribution:
By City:
  Seattle: 713 employees
  Los Angeles: 1,312 employees
  Denver: 598 employees
  Austin: 706 employees
By Job 

In [29]:
# Chat GPT
import pandas as pd
import itertools
import numpy as np
from scipy.optimize import linprog

# Define parameters
cities = ['Seattle', 'Los Angeles', 'Denver', 'Austin']
job_categories = ['Engineer', 'Scientist', 'Accountant']
levels = [1, 2, 3, 4, 5]
avg_cost = 8000
budget = 5_000_000

# Create all combinations
combinations = list(itertools.product(cities, job_categories, levels))
df = pd.DataFrame(combinations, columns=['City', 'Positions', 'Level'])

# Assign people data
df['People'] = [
    100, 200, 200, 50, 5,
    25, 45, 55, 15, 3,
    3, 4, 5, 2, 1,
    200, 300, 350, 150, 15,
    50, 80, 95, 30, 6,
    9, 12, 9, 4, 2,
    80, 150, 150, 50, 5,
    30, 45, 55, 15, 3,
    3, 4, 5, 2, 1,
    120, 130, 140, 80, 10,
    60, 50, 50, 30, 5,
    6, 10, 9, 4, 2,
]

# Generate possible move options (same position and level, different cities)
move_options = []
costs = []

for idx_from, row_from in df.iterrows():
    for idx_to, row_to in df.iterrows():
        if (
            row_from['City'] != row_to['City'] and
            row_from['Positions'] == row_to['Positions'] and
            row_from['Level'] == row_to['Level']
        ):
            max_moves = min(row_from['People'], row_to['People'])  # conservatively limit
            move_options.append((idx_from, idx_to, max_moves))
            costs.append(avg_cost)

#print(f'Max Moves: {move_options}')
            
# Objective function (maximize moves, so minimize -1 * moves)
c = [-1] * len(move_options)

# Budget constraint
A_ub = [costs]
b_ub = [budget]

# Bounds: 0 to max_moves
bounds = [(0, max_mv) for _, _, max_mv in move_options]

# Solve the optimization
res = linprog(c=c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# Collect flat records directly for DataFrame
move_records = []

if res.success:
    for i, val in enumerate(res.x):
        if val > 0:
            idx_from, idx_to, _ = move_options[i]
            row_from = df.iloc[idx_from]
            row_to = df.iloc[idx_to]
            move_records.append({
                'From_City': row_from['City'],
                'From_Position': row_from['Positions'],
                'From_Level': row_from['Level'],
                'From_People': row_from['People'],
                'To_City': row_to['City'],
                'To_Position': row_to['Positions'],
                'To_Level': row_to['Level'],
                'To_People': row_to['People'],
                'Moves': int(round(val)),
                'Cost': int(round(val * avg_cost))
            })

# Convert to DataFrame
moves_df = pd.DataFrame(move_records)
moves_df

,From_City,From_Position,From_Level,From_People,To_City,To_Position,To_Level,To_People,Moves,Cost
0,Austin,Engineer,4,80,Los Angeles,Engineer,4,150,28,224000
1,Austin,Engineer,4,80,Denver,Engineer,4,50,50,400000
2,Austin,Engineer,5,10,Seattle,Engineer,5,5,5,40000
3,Austin,Engineer,5,10,Los Angeles,Engineer,5,15,10,80000
4,Austin,Engineer,5,10,Denver,Engineer,5,5,5,40000
5,Austin,Scientist,1,60,Seattle,Scientist,1,25,25,200000
6,Austin,Scientist,1,60,Los Angeles,Scientist,1,50,50,400000
7,Austin,Scientist,1,60,Denver,Scientist,1,30,30,240000
8,Austin,Scientist,2,50,Seattle,Scientist,2,45,45,360000
9,Austin,Scientist,2,50,Los Angeles,Scientist,2,80,50,400000
